## Select by length

此示例选择器根据长度选择要使用的示例。当您担心构建的提示会超过上下文窗口的长度时，这非常有用。对于较长的输入，它将选择较少的示例来包含，而对于较短的输入，它将选择更多的示例。

In [1]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


# 创建一个反义词的任务示例
examples = [
    {"input": "开心", "output": "伤心"},
    {"input": "高", "output": "矮"},
    {"input": "粗", "output": "细"},
    {"input": "下雨天", "output": "晴天"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

example_selector = LengthBasedExampleSelector(
    # 可供选择的示例
    examples=examples,
    # 示例的提示词模板
    example_prompt=example_prompt,
    # 示例的最大长度
    # 长度由下面get_text_length函数计算
    max_length=25
    # 用于获取字符串长度的函数，使用
    # 确定要包含哪些示例。被注释掉是因为
    # 如果未指定，则将其作为默认值提供。
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)

# 创建一个动态提示词模板
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="Input: {adjective}\nOutput: ",
    input_variables=["adjective"],
)


In [2]:
# 输入量较小时，给出所有的示例
print(dynamic_prompt.format(adjective="happy"))

给出每个输入的反义词

Input: 开心
Output: 伤心

Input: 高
Output: 矮

Input: 粗
Output: 细

Input: 下雨天
Output: 晴天

Input: happy
Output: 


In [3]:
# 输入量较大时，给出一个示例
long_string = " ".join(["happy"] * 20)
print(dynamic_prompt.format(adjective=long_string))


给出每个输入的反义词

Input: 开心
Output: 伤心

Input: happy happy happy happy happy happy happy happy happy happy happy happy happy happy happy happy happy happy happy happy
Output: 


In [4]:
# 添加一个新的示例
new_example = {"input": "胖", "output": "瘦"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="热情"))


给出每个输入的反义词

Input: 开心
Output: 伤心

Input: 高
Output: 矮

Input: 粗
Output: 细

Input: 下雨天
Output: 晴天

Input: 胖
Output: 瘦

Input: 热情
Output: 


In [5]:
# 与本地部署的大模型配合使用
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"
chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.7)

output_parser = StrOutputParser()

chain = dynamic_prompt | chat | output_parser

chain.invoke({"adjective": "热情似火"})

'冷淡如冰'

## 最大余弦相似度的嵌入示例
MaxMarginalRelevanceExampleSelector 根据与输入最相似的示例组合来选择示例，同时还针对多样性进行优化。它通过查找与输入具有最大余弦相似度的嵌入示例来实现这一点，然后迭代地添加它们，同时惩罚它们与已选择示例的接近程度。

```
pip install sentence-transformers
pip install faiss-cpu
```

In [7]:
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores import FAISS
import sys
sys.path.append("../")

from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# 确保提供本地模型路径
embeddings_path = "/Users/libing/kk_LLMs/bge-large-zh-v1.5"

# 在 HuggingFaceEmbeddings 中明确设置本地路径
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)


# 示例 Prompt 模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 示例数据
examples = [
    {"input": "开心", "output": "伤心"},
    {"input": "高", "output": "矮"},
    {"input": "粗", "output": "细"},
    {"input": "下雨天", "output": "晴天"},
]

# 使用 Max Marginal Relevance Example Selector
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=FAISS,
    k=2
)

# FewShot Prompt 模板
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="Input: {adjective}\nOutput: ",
    input_variables=["adjective"],
)

# 测试 Prompt
formatted_prompt = mmr_prompt.format(adjective="sad")
print(formatted_prompt)


给出每个输入的反义词

Input: 开心
Output: 伤心

Input: 粗
Output: 细

Input: sad
Output: 


In [8]:
chain = mmr_prompt | chat | output_parser
chain.invoke({"adjective": "sad"})

'happy'

In [9]:
chain.invoke({"adjective": "担心"})

'放心'

## 通过n-gram重叠选择

NGramOverlapExampleSelector 根据 ngram 重叠得分，根据与输入最相似的示例来选择示例并对其进行排序。 ngram 重叠分数是 0.0 到 1.0 之间的浮点数（含 0.0 和 1.0）。

选择器允许设置阈值分数。 ngram 重叠分数小于或等于阈值的示例被排除。默认情况下，阈值设置为 -1.0，因此不会排除任何示例，只会对它们重新排序。将阈值设置为 0.0 将排除与输入没有 ngram 重叠的示例。

In [13]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 构建翻译示例
examples = [
    {"input": "See spot run.", "output": "请参阅现场运行。"},
    {"input": "My dog barks.", "output": "我的狗在叫。"},
    {"input": "Cat can run.", "output": "猫会跑。"},
]

example_selector = NGramOverlapExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    threshold=0.0
    # 对于负阈值：择器按 ngram 重叠分数对示例进行排序，并且不排除任何示例。
    # 对于大于 1.0 的阈值：选择器排除所有示例，并返回一个空列表。
    # 对于阈值等于 0.0:选择器按 ngram 重叠分数对示例进行排序，并排除那些与输入没有 ngram 重叠的内容。
)

example_selector.select_examples({"input": "My car is red and can run fast."})

[{'input': 'My dog barks.', 'output': '我的狗在叫。'},
 {'input': 'Cat can run.', 'output': '猫会跑。'}]

In [16]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="一个中英文翻译案例",
    suffix="Input: {sentence}\nOutput: ",
    input_variables=["sentence"],
)

# 一个与猫会跑有大量重叠，并且与狗会叫没有重叠的例子
dynamic_prompt.format(sentence="we car is red and can run fast.")

'一个中英文翻译案例\n\nInput: Cat can run.\nOutput: 猫会跑。\n\nInput: we car is red and can run fast.\nOutput: '

In [23]:
example_selector.threshold = 0.009
dynamic_prompt.format(sentence="cat can play fetch.")


'一个中英文翻译案例\n\nInput: Cat can run.\nOutput: 猫会跑。\n\nInput: cat can play fetch.\nOutput: '

In [24]:
chain = dynamic_prompt | chat | output_parser
chain.invoke({"sentence": "cat can play fetch."})


'猫可以玩接球游戏。'